# Gradient Boosting

As we have seen, Random Forest is an ensemble learning method, which predicts by combining outputs from individual trees. It uses deep decision trees (low bias, high variance) and combines them, which reduces the variance. 
Another approach is to combine weak learners (high bias, low variance). In case of decision trees, this means that shallow trees are used. It reduces the high bias by adding a classifier at a time, the next classifier is trained to improve the previous trained ensemble.

More formally: Suppose we have a weak learner $W(x, \omega)$, where $x$ is our input vector and $\omega$ the parameters of the model. In case of trees, $\omega$ is:

$$
\omega = \{R_p, c_p\} \Big\rvert_{p=1}^{p=P}
$$

$P$ is the number of partitions, $R_p$ the region and $c_p$ its value for a given partition. The goal is to have after $M$ iterations steps, a function $f_M$ which adds multiple weak models $f_m$ together.
At each step $m$, we improve the previous model by adding a weak learner with a learning rate $\rho_m$ (also known as shrinkage parameter):

$$
f_m(x) = f_{m-1}(x) + \rho_m W_m(x, \omega_m)
$$

The initial step $f_0$ can be set to 0 or to the average of the output vector $y$.
Motivated by gradient descent, to find the parameters for the weak learner $W_m$, we take the gradient of the loss function $J$ to calculate the direction of our steps. The gradient $r_{im}$ (also known as residual) is:

$$
r_{im} = -\Big( \frac{\partial J(y^{(i)}, f(x^{(i)}))}{\partial f(x^{(i)}) } \Big)_{f(x^{(i)})=f_{m-1}(x^{(i)})}
$$

We use this gradient to calculate the parameters for the weak learner $W_m$ by fitting a model to the negative gradient:

$$
\omega_m = \operatorname*{arg\,min}_{\omega} \sum_{i=1}^{N} J(r_{im}, W(x^{(i)}, \omega))
$$

The learning rate $\rho_m$ can be either fixed or can be computed:

$$
\rho_m = \operatorname*{arg\,min}_{\rho} \sum_{i=1}^{N} J(y^{(i)}, f_{m-1}(x) + \rho W_m(x, \omega_m))
$$

## Summarization

1. Initialize $f_0(x) = 0$ or $f_0 = \frac{1}{N} \sum y^{(i)}$

2. For $m = 1$ to M:
    
    (a) Compute: 
    $$
    r_{im} = -\Big( \frac{\partial J(y^{(i)}, f(x^{(i)}))}{\partial f(x^{(i)}) } \Big)_{f(x^{(i)})=f_{m-1}(x^{(i)})}
    $$
    
    (b) Fit model to residuals:
    $$
    \omega_m = \operatorname*{arg\,min}_{\omega} \sum_{i=1}^{N} J(r_{im}, W(x^{(i)}, \omega))
    $$
    
    (c) Choose fixed step size $\rho_m$ or compute it
    
    (d) Take the step:
    $$
    f_m(x) = f_{m-1}(x) + \rho_m W_m(x, \omega_m)
    $$
    
3. Output: 

$$
f_m(x) = f_0(x) + \sum_{m=1}^M \rho_m W_m(x, \omega_m)
$$